### First, we're going to load up the appropriate libraries

In [2]:
import os
import requests
import boto3
import json
import pandas as pd
from datetime import datetime
from time import sleep
import pprint

### Next, we set up Domino API credentials.

Since we are running this Python code inside of a Domino workspace, we can pull these credentials directly from the Environment Variables. If you run this code outside of Domino, you will need to manually populate these variables.

In [3]:
dominoAPIHost = os.environ["DOMINO_API_HOST"].strip()
dominoAPIKey = os.environ["DOMINO_USER_API_KEY"].strip()
dominoUserName = os.environ["DOMINO_PROJECT_OWNER"].strip()
dominoProjectName = os.environ["DOMINO_PROJECT_NAME"].strip()

### Now we will set up some ECR variables.

Keep in mind that "domino-sagemaker-exports" is an example ECR repository, but you will need to create your own ECR repository in your AWS account.

In [9]:
ecrRegion = os.environ["AWS_DEFAULT_REGION"].strip()
ecrRepository = "domino-sagemaker-exports"

### With configuration out of the way, we can now paste in some boilerplate code to wrap around the Domino SageMaker export API

In [10]:
# Generate ECR login details for Domino's API
def generateECRPushDetails(repository, tag):
    from base64 import b64decode
    
    ecrAccount = boto3.client("sts", region_name=ecrRegion).get_caller_identity().get("Account")
    ecrToken = boto3.client("ecr", region_name=ecrRegion).get_authorization_token(registryIds=[ecrAccount])
    ecrRegistry = ecrToken["authorizationData"][0]["proxyEndpoint"].lstrip("https://").lstrip("http://")
    ecrAuth = b64decode(ecrToken["authorizationData"][0]["authorizationToken"]).decode("utf-8").split(":")
    
    pushData = {
        "registryUrl": ecrRegistry,
        "repository": repository,
        "tag": tag,
        "username": ecrAuth[0],
        "password": ecrAuth[1]
    }
    
    return pushData

# Domino API Request Base Method to simplify making Domino API calls
def dominoRequest(urlPath, requestType, json = None):
    url = "{DOMINO_HOST}{URL_PATH}".format(
        DOMINO_HOST = dominoAPIHost,
        URL_PATH = urlPath
    )
    
    session = requests.Session()
    session.headers.update({
        "X-Domino-Api-Key": dominoAPIKey,
        "Content-Type": "application/json",
        "Accept": "application/json"
    })
    response = None
    if requestType == "GET":
        response = session.get(url)
    elif requestType == "POST":
        response = session.post(url, json = json)
    
    response.raise_for_status()

    return response

# Domino API Wrapper for /v4/models/<MODEL_ID>/<MODEL_VERSION_ID>/exportImageForSagemaker
# This API request exports a Domino model to ECR as a SageMaker compatible format
def dominoSagemakerExport(modelID, modelVersion):
    modelVersionID = next(iter([modelVersionInfo.get("_id", None) for modelVersionInfo in dominoModelVersions(modelID) if modelVersionInfo.get("metadata", {}).get("number", None) == modelVersion]), None)
    exportStatus = None

    if modelVersionID:
        urlPath = "/v4/models/{MODEL_ID}/{MODEL_VERSION_ID}/exportImageForSagemaker".format(
            MODEL_ID = modelID,
            MODEL_VERSION_ID = modelVersionID
        )

        exportTag = "domino-{USERNAME}-{PROJECT_NAME}-{MODEL_ID}-{MODEL_VERSION}".format(
            USERNAME = dominoUserName,
            PROJECT_NAME = dominoProjectName,
            MODEL_ID = modelID,
            MODEL_VERSION = modelVersion
        )

        jsonData = generateECRPushDetails(ecrRepository, exportTag)

        exportStatus = json.loads(dominoRequest(urlPath, "POST", jsonData).text)
        exportStatus["tag"] = exportTag
        exportStatus["url"] = "https://{REGISTRY_URL}/{REPOSITORY}:{TAG}".format(
            REGISTRY_URL = jsonData["registryUrl"],
            REPOSITORY = jsonData["repository"],
            TAG = jsonData["tag"]
        )
    
    return exportStatus

## The following API calls are not required to publish a Domino model to ECR, but are going to be useful to list all of the running models for this Domino project

# Domino API Wrapper for /v1/projects/<USERNAME>/<PROJECT_NAME>/models
# This API Request returns all of the models associated with a particular Domino project
def dominoProjectModels():
    urlPath = "/v1/projects/{USERNAME}/{PROJECT_NAME}/models".format(
        USERNAME = dominoUserName,
        PROJECT_NAME = dominoProjectName
    )
    
    return json.loads(dominoRequest(urlPath, "GET").text).get("data", [])

# Domino API Wrapper for /v1/models/<MODEL_ID>/versions
# This API requests returns all of the Domino model versions for a particular model
def dominoModelVersions(modelID):
    urlPath = "/v1/models/{MODEL_ID}/versions".format(
        MODEL_ID = modelID
    )

    return json.loads(dominoRequest(urlPath, "GET").text).get("data", [])

# Domino API Wrapper for /v4/models/<EXPORT_ID>/getExportLogs
# This API requests get the logs for a Domino model export
def dominoModelExportLogs(exportID):
    urlPath = "/v4/models/{EXPORT_ID}/getExportLogs".format(
        EXPORT_ID = exportID
    )
    
    return json.loads(json.loads(dominoRequest(urlPath, "GET").text).get("logs", None))

# Domino API Wrapper for /v4/models/<EXPORT_ID>/getExportImageStatus
# This API request gets that status of a Domino model export
def dominoModelExportStatus(exportID):
    urlPath = "/v4/models/{EXPORT_ID}/getExportImageStatus".format(
        EXPORT_ID = exportID
    )
    
    return json.loads(dominoRequest(urlPath, "GET").text)


### Now, we just need to select which model we want to export.

We can visualize the models associated with our current Domino Project with the following code

In [11]:
dominoModels = []
for model in dominoProjectModels():
    for modelVersion in dominoModelVersions(model["id"]):
        dominoModel = {
            "Name": model["name"],
            "Model ID": model["id"],
            "Version": modelVersion["metadata"]["number"],
            "Published": datetime.fromtimestamp(float(modelVersion["metadata"]["created"]) / 1000),
            "File": modelVersion["file"],
            "Function": modelVersion["function"]
        }
        dominoModels.append(dominoModel)

models = pd.DataFrame.from_dict(dominoModels).sort_index()
models

,File,Function,Model ID,Name,Published,Version
0,code/smallPredict.py,predict,5ed7dadecdfc18044c36d381,Churn Prediction,2020-06-11 17:33:19.346,2
1,code/predict.py,predict,5ed7dadecdfc18044c36d381,Churn Prediction,2020-06-03 17:16:15.049,1


The Domino SageMaker Export API will require us to use the Model ID and Model Version. In this case, we will select model ID "5ed7dadecdfc18044c36d381" and model version 2.

### Finally, we call our model export API with the above parameters and wait for it to finish.

In [12]:
modelExport = dominoSagemakerExport("5ed7dadecdfc18044c36d381", 2)

# How often, in seconds, to check the status of the model export
SLEEP_TIME_SECONDS = 10

while modelExport:
    status = dominoModelExportStatus(modelExport["exportId"]).get("status", None)
    if status:
        print(status)

        if status not in ["complete", "failed"]:
            sleep(SLEEP_TIME_SECONDS)
        else:
            break

preparing
preparing
preparing
preparing
preparing
preparing
preparing
complete


### Let's take a look at the export logs formore details

In [13]:
logs = dominoModelExportLogs(modelExport["exportId"])
pprint.pprint(logs)

{'lines': [{'content': 'Pulling and pushing Docker image '
                       '100.97.86.247:5000/dom-mdl-5ed7dadecdfc18044c36d381-v2:2020611173319_t7msUfeb',
            'timestamp': '20200825T184311.843Z'},
           {'content': '', 'timestamp': '20200825T184311.843Z'},
           {'content': 'Building Sagemaker Image...',
            'timestamp': '20200825T184313.425Z'},
           {'content': '======================================================================',
            'timestamp': '20200825T184313.481Z'},
           {'content': 'Starting', 'timestamp': '20200825T184313.481Z'},
           {'content': '======================================================================',
            'timestamp': '20200825T184313.481Z'},
           {'content': 'Preparing project and dependencies',
            'timestamp': '20200825T184313.753Z'},
           {'content': 'Linking project to /mnt/imarchenko/CustomerChurn',
            'timestamp': '20200825T184359.377Z'},
           {'co